In [1]:
import os
import numpy as np
import music21 as ms
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Activation,Embedding

Using TensorFlow backend.


# Generating data

## return a list consisting of pitch id of note and note id separated by dot in a chord 

## if the file has several instruments it takes only one in account

In [2]:
path="data"
def datagen(files=48):
    notes=[]
    j=os.listdir("data")
    for i in range(files):
        midi=ms.converter.parse(path+"//"+j[i])
        
        notes_file=ms.instrument.partitionByInstrument(midi)
        
        if notes_file:
            notes_file=notes_file[0].recurse()
            
        else:
            notes_file=notes_file.flat.notes
        
        for k in notes_file:
            if isinstance(k,ms.note.Note):
                notes.append(str(k.pitch))
            if isinstance(k,ms.chord.Chord):
                notes.append('.'.join(str(n) for n in k.normalOrder))
        print(i,end=" ")       
    return notes
        
        

In [3]:

#48 total files
note=datagen()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 

#  Mapping of pitches(string data) to numbers

# And creating dataset with lookback 100


In [4]:
pitchesname=sorted(set(note))    
# mapp=dict((note, number) for number, note in enumerate(pitchesname))

# def mapping(note,lookback=100):
#     x=[]
#     y=[]
    
#     for i in range(0,len(note)-lookback,1):
# #         print(i,end=" ")
#         x.append(np.array([mapp[j] for j in note[i:i+lookback]]))
#         y.append(np.array([mapp[note[i+lookback]]]))
#     x=np.array(x)
#     y=np.array(y)
#     x=x.reshape((x.shape[0],lookback))
    
#     return x,y,len(pitchesname)
        

    
    

In [5]:
# data_x,data_y,n_vocab=mapping(note)
# data_x.shape,data_y.shape,n_vocab

In [6]:
# import pickle
# input_file=open("input_file","wb")
# output_file=open("output_file","wb")
# pickle.dump(data_x,input_file)
# pickle.dump(data_y,output_file)
# output_file.close()
# input_file.close()

In [7]:
# def create_model(input_shape1,n_voacb):
#     model=Sequential()
#     model.add(Embedding(input_dim=n_voacb+1,output_dim=256))
# #     model.add(LSTM(256,return_sequences=True))
# #     model.add(Dropout(0.3))
# #     model.add(LSTM(512, return_sequences=True))
# #     model.add(Dropout(0.3))
#     model.add(LSTM(256))
#     model.add(Dense(256))
#     model.add(Dropout(0.3))
#     model.add(Dense(n_vocab))
#     model.add(Activation('softmax'))
#     model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
#     return model

In [8]:
# model=create_model(data_x.shape[1:],n_vocab)
# n_vocab

In [9]:
# model.fit(data_x[:500],data_y[:500],epochs=38)

In [10]:
model=keras.models.load_model("musicmodel.h5")

In [17]:
dmapp=dict((number,note) for number, note in enumerate(pitchesname))
def generate_notes(model,start,n=10):
    pred_notes=[]
    start=list(start)
    k=1
    for i in range(n):
        s=np.reshape(start,(1,100))
        if(k%10==0):
            s=np.random.randint(0,317,size=(1,100))
        pred=np.argmax(model.predict(s),axis=1)
        result=dmapp[pred[0]]
        pred_notes.append(result)
        start.append(pred[0])
        start=start[1:]
    return pred_notes

In [18]:
# n=np.random.randint(0,len(data_x)-1)
arr=np.random.randint(0,317,size=(100))
gen_notes=generate_notes(model,arr,n=50)

In [19]:
arr

array([106, 244, 140,  87,   1, 273, 128, 244, 116, 238, 124, 229, 294,
       268,  61, 128, 286,  77, 144,  16, 251,  41, 274,  32, 314, 131,
       255,  62, 237,   6,  33, 210,  76,  17,  50, 215,  79,  18, 202,
       260, 314,  92, 280, 212, 143, 282, 256,  44,  61,  96, 295, 277,
       286, 110, 290,  91, 145,  26, 104,  39, 150, 253, 211,  96, 220,
       216,  64, 154, 175, 219, 308, 264, 227,  12, 182,  80, 214, 241,
       229,  91, 184, 218, 251, 190, 260, 209,  43, 142, 256, 143, 231,
        87,  90,  12, 248, 295, 241, 225, 233, 243])

In [20]:
def genstream(pred_notes):
    offset=0
    gen_stream=[]
    for i in pred_notes:
        if (("." in i) or (i.isdigit())):
            notesinc=i.split(".")
            note=[]
            for j in notesinc:
                newnote=ms.note.Note(int(j))
                newnote.storedInstrument=ms.instrument.Banjo()
                note.append(newnote)
            newchord=ms.chord.Chord(note)
            newchord.offset=offset
            gen_stream.append(newchord)
        else:
            newnote=ms.note.Note(i)
            newnote.storedInstrument=ms.instrument.Bagpipes()
            newnote.offset=offset
            gen_stream.append(newnote)
        
        offset+=0.5
    return gen_stream
            

In [21]:
gen_music=genstream(gen_notes)

In [22]:
gen_music

[<music21.note.Note A>,
 <music21.note.Note F#>,
 <music21.note.Note E->,
 <music21.note.Note F#>,
 <music21.note.Note A>,
 <music21.note.Note F#>,
 <music21.note.Note E->,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note E->,
 <music21.note.Note A>,
 <music21.note.Note C>,
 <music21.note.Note G#>,
 <music21.note.Note D>,
 <music21.note.Note E->,
 <music21.note.Note E->,
 <music21.note.Note D>,
 <music21.note.Note C>,
 <music21.note.Note C>,
 <music21.note.Note G#>,
 <music21.note.Note G#>,
 <music21.note.Note E->,
 <music21.note.Note G#>,
 <music21.note.Note F#>,
 <music21.note.Note G#>,
 <music21.note.Note G#>,
 <music21.note.Note G>,
 <music21.note.Note E->,
 <music21.note.Note G#>,
 <music21.note.Note F#>,
 <music21.note.Note G#>,
 <music21.note.Note G#>,
 <music21.note.Note G#>,
 <music21.note.Note E->,
 <music21.note.Note A>,
 <music21.note.Note C>,
 <music21.note.Note G#>,
 <music21.note.Note A>,
 <music21.note.Note G#>,
 <music21.no

In [23]:
midi_stream=ms.stream.Stream(gen_music)
midi_stream.write('midi',fp='test_output3.mid')

'test_output3.mid'